<a href="https://colab.research.google.com/github/ArvindSharma18/assAIgnment/blob/main/DPO_Book_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset Preprocessing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/My_Book_Reviews_Dataset.csv",index_col=False)

In [ ]:
def create_orpo_instruction_prompt_template(row):
  prompt = f"""<|start_header_id|>system<|end_header_id|> You are an helpful Assistant for a Book Reviewer. You are trained to create a draft Book Review for your Reviwer. Your book review should be close to your Reviwer's style to ease his job.<|eot_id|><|start_header_id|>user<|end_header_id|> Following are the Details for the Book review:\n Title: {row['Title']} \n Author: {row['Author']} \n Reviewer Rating: {row['My Rating']} \n Average Rating: {row['Average Rating']} \n Publisher: {row['Publisher']} \n Number of Pages: {row['Number of Pages']} \n Year Published: {row['Year Published']} \n Provide a Book Review Similar to your Reviwer<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

  return prompt


In [ ]:
df['orpo_instructions'] = df.apply(create_orpo_instruction_prompt_template, axis =1)

In [ ]:
inputs = tokenizer(
df['orpo_instructions'].tolist(), return_tensors = "pt", padding= True).to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 1024)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
df['model_output'] = tokenizer.batch_decode(outputs, skip_special_tokens = True)

In [ ]:
df['model_review'] = [str(review.split("Provide a Book Review Similar to your Reviwer")[1]) for review in df['model_output']]

In [ ]:
def orpo_dataset_format(row):
  row["prompt"] = row['Prompt_Instruction']
  row["chosen"] = row['My Review']
  row["rejected"] = row['model_review']

  return row

In [ ]:
df = df.apply(orpo_dataset_format,axis =1)


In [ ]:
df = pd.read_csv("/content/My_Book_Reviews_Dataset_ORPO.csv",index_col=False)

In [ ]:
df = df[['prompt', 'chosen','rejected']]

In [ ]:
df.to_csv("/content/My_Book_Reviews_Dataset_ORPO.csv",index = False)

### DPO Traning

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "ArvindSharma18/llama-3-8b-Instruct-bnb-4bit-book-review", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch_dtype,
        bnb_4bit_use_double_quant=True,
    )

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("csv", data_files="/content/My_Book_Reviews_Dataset_ORPO.csv",split = "train")

In [ ]:
# Enable reward modelling stats
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [ ]:
max_seq_length = 4096

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 2e-3,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
    beta = 0.1,
    train_dataset = dataset,
    # eval_dataset = raw_datasets["test"],
    tokenizer = tokenizer,
    max_length = 2048,
    max_prompt_length = 1024,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [ ]:
#del trainer, model
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 64 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 48
 "-____-"     Number of trainable parameters = 4,718,592
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 